# Backup Validation and DBCC CHECKDB using Rubrik Live Mount

## Backup Validatation of a database using Live Mount

Live mount allows for near instant recovery of a database. If a database restore/export normally takes hours, then live mounting a database will take a few minutes. Live Mount does a full recovery of a database to either the same SQL Server Instance with a different database name or another SQL Server Instance with the same or different database name. The recovery of the database is much faster, because Rubrik does not need to copy the contents of the backup from the Rubrik Cluster back to the SQL Server. All of the recovery work is done on the Rubrik cluster itself. Then the database files are presented to the SQL Server Instance via a secure SMB3 share that is only accessible by the machine the share is mounted to.

Live Mounting a database is great for a lot of different use cases:
- DBA Backup validation testing
- Object level recovery
- Developer testing
- DevOps Automation
- Reporting databases
- Database migration application smoke test validation.

A key parameter is RecoveryDateTime. All dates in Rubrik are stored in UTC format. This parameter is expecting a fully qualified date and time in UTC format. example value is 2018-08-01T02:00:00.000Z. In the example below, we are pulling the latest recovery point that Rubrik knows about.

**This article serves as a way to demonstrate how to use Live Mount for Backup Validation.**

### Set Variables Which will be used throughout the rest of this demo

In [ ]:
# Set Variables
$Server = $Rubrik.Server.cdm02
$Token = $Rubrik.token.cdm02
$SourceSQLServerInstance = "rp-sql19s-001.perf.rubrik.com"
$SourceDatabaseName = "AdventureWorks2019"
$TargetSQLServerInstance = "rp-sql19s-001.perf.rubrik.com"
$MountedDatabaseName = "AdventureWorks2019_LiveMount"

#### Connect to the Rubrik Cluster

In [ ]:
#Connect-Rubrik with an API Token
Connect-Rubrik -Server $Server -Token $Token

#### Get details about the database from the Rubrik Cluster

In [ ]:
# Get database information from Rubrik
$RubrikDatabase = Get-RubrikDatabase -Name $SourceDatabaseName -ServerInstance $SourceSQLServerInstance
$RubrikDatabase | Format-List *

#### Mount the database to a SQL Server

In [ ]:
#Mount a database to a SQL Server
$TargetInstance = Get-RubrikSQLInstance -ServerInstance $TargetSQLServerInstance
$RubrikRequest = New-RubrikDatabaseMount -id $RubrikDatabase.id `
	-TargetInstanceId $TargetInstance.id `
	-MountedDatabaseName $MountedDatabaseName `
	-recoveryDateTime (Get-date (Get-RubrikDatabase -id $RubrikDatabase.id).latestRecoveryPoint) `
    -Confirm:$false
Get-RubrikRequest -id $RubrikRequest.id -Type mssql -WaitForCompletion    

A Live mount of a database is the equivolent to doing a T-SQL Restore with your native backups. SQL Server has recovered the snapshot via the SQL Server VSS Writer, and if applicable, rolled the database forward to a point in time chosen by the user. This means we have applied all transactions from the time the snapshot has happened until the point in time chosen. Once a database has been Live Mounted to a SQL Server, the database is ready for any read/write query you would like to run. 